In [1]:
%matplotlib widget

In [2]:
import pandas as pd
import numpy as np
from glob import glob
import os
import datetime
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Input, Concatenate, Dot, Add, ReLU, Activation
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow import keras

In [3]:
import matplotlib
import matplotlib.font_manager as fm
fm.get_fontconfig_fonts()
font_location = '/usr/share/fonts/truetype/nanum/NanumGothicCoding.ttf'
#font_location = '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc'
# font_location = 'C:/Windows/Fonts/NanumGothic.ttf' # For Windows
fprop = fm.FontProperties(fname=font_location)

In [80]:
prefix = 'han'
gan_files = [ '가평_gan.xlsx', '의암호_gan.xlsx', '서상_gan.xlsx']
ori_files = [ '가평_org.xlsx', '의암호_org.xlsx', '서상_org.xlsx']

In [407]:
prefix = 'nagdong2'
gan_files = [ '도개_gan.xlsx', '신암_gan.xlsx', '회상_gan.xlsx']
ori_files = [ '도개_org.xlsx', '신암_org.xlsx', '회상_org.xlsx']

In [4]:
prefix = 'nagdong3'
gan_files = [ '도개3_gan.xlsx', '신암3_gan.xlsx']
ori_files = [ '도개3_org.xlsx', '신암3_org.xlsx']

In [336]:
prefix = 'nagdong4'
gan_files = [ '도개4_gan.xlsx', '신암4_gan.xlsx']
ori_files = [ '도개4_org.xlsx', '신암4_org.xlsx']

In [480]:
prefix = 'geum'
gan_files = [ '현도_gan.xlsx', '대청호_gan.xlsx', '장계_gan.xlsx']
ori_files = [ '현도_ori.xlsx', '대청호_ori.xlsx', '장계_ori.xlsx']

In [149]:
prefix = 'geum2'
gan_files = [ '현도2_gan.xlsx', '대청호2_gan.xlsx', '장계2_gan.xlsx']
ori_files = [ '현도2_ori.xlsx', '대청호2_ori.xlsx', '장계2_ori.xlsx']

In [313]:
prefix = 'yeong'
gan_files = [ '나주_gan.xlsx', '용봉_gan.xlsx', '우치_gan.xlsx']
ori_files = [ '나주_org.xlsx', '용봉_org.xlsx', '우치_gan.xlsx']

In [337]:
folder = 'data'
def save_gan_data(gan_dfs, ori_dfs, gan_files, ori_files, train_mean, train_std, prefix):
    for i in range(len(gan_dfs)):
        path = os.path.join(folder, gan_files[i])
        gan_dfs[i].to_excel(path, index=False)
        print(path)
        path = os.path.join(folder, ori_files[i])
        ori_dfs[i].to_excel(path, index=False)
    path = os.path.join(folder, prefix + '_std.xlsx')
    train_std.to_frame(name='std').to_excel(path)
    path = os.path.join(folder, prefix + '_mean.xlsx')
    train_mean.to_frame(name='mean').to_excel(path)

def load_gan_data(gan_files, ori_files, prefix):
    gan_dfs = []
    ori_dfs = []
    for i in range(len(gan_files)):
        path = os.path.join(folder, gan_files[i])
        gan_dfs.append(pd.read_excel(path))
        print(path)
        path = os.path.join(folder, ori_files[i])
        ori_dfs.append(pd.read_excel(path))
    path = os.path.join(folder, prefix + '_std.xlsx')
    train_std = pd.read_excel(path, index_col=0).loc[:, 'std']
    path = os.path.join(folder, prefix + '_mean.xlsx')
    train_mean = pd.read_excel(path, index_col=0).loc[:, 'mean']
    return gan_dfs, ori_dfs, train_mean, train_std

In [338]:
gan_dfs, ori_dfs, train_mean, train_std = load_gan_data(gan_files, ori_files, prefix=prefix)
for i in range(len(gan_dfs)):
    gan_dfs[i].columns = gan_dfs[i].columns+'%d'%(i+1)
    ori_dfs[i].columns = ori_dfs[i].columns+'%d'%(i+1)

data/도개4_gan.xlsx
data/신암4_gan.xlsx


In [271]:
gan_dfs[2]

,tmpr_value3,ph_value3,do_value3,ec_value3,toc_value3,총질소_값3,총인_값3,클로로필-a_값3
0,-1.096902,0.758452,0.822259,-0.075157,-0.344853,0.275620,0.062420,-0.152076
1,-1.139439,0.526553,0.766793,-0.099507,-0.389877,0.283430,-0.027359,-0.220411
2,-1.324523,0.522525,0.704634,-0.202995,-0.528889,0.262529,-0.068776,0.082653
3,-1.295462,0.524686,0.830331,0.100130,-0.376141,0.190097,-0.022085,-0.000649
4,-1.213552,0.770288,0.937213,-0.199829,-0.230602,0.410256,0.061129,-0.096691
...,...,...,...,...,...,...,...,...
68803,-0.570429,-0.086093,0.581647,-0.034047,-0.546348,-0.619780,-1.060085,-0.646295
68804,-0.576506,-0.086093,0.641735,-0.209620,-0.639205,-0.633176,-1.053605,-0.626142
68805,-0.582583,-0.086093,0.689536,-0.313739,-0.770294,-0.678631,-1.040449,-0.608782
68806,-0.588659,-0.086093,0.688196,-0.312086,-0.747662,-0.725383,-1.046929,-0.590625


In [195]:
68664*0.2

13732.800000000001

In [340]:
cutoff = -3000 # 낙동강
#cutoff = -3720 # 영산강
gan_dfs[0] = gan_dfs[0][:cutoff]
gan_dfs[1] = gan_dfs[1][:cutoff]
#gan_dfs[2] = gan_dfs[2][:cutoff]
ori_dfs[0] = ori_dfs[0][:cutoff]
ori_dfs[1] = ori_dfs[1][:cutoff]
#ori_dfs[2] = ori_dfs[2][:cutoff]

In [154]:
n_len = len(gan_dfs[0])
n_val_len = n_len//10
n_len, n_val_len

(65664, 6566)

In [436]:
49304-3000

46304

In [359]:
len(gan_dfs[0])

46504

In [410]:
file_names = []
file_names = ['보_구미보_2016.xlsx', '보_구미보_2017.xlsx', '보_구미보_2018.xlsx', '보_구미보_2019.xlsx']
folder = 'data'
bo_dfs = []
for file_name in file_names:
    path = os.path.join(folder, file_name)
    print(path)
    bo_dfs.append(pd.read_excel(path).iloc[:, 2:4])
bo_df = pd.concat(bo_dfs, axis=0)

bo_mean = bo_df.mean()
bo_std = bo_df.std()
bo_df = (bo_df - bo_mean)/bo_std

bo_df.reset_index(drop=True, inplace=True)

data/보_구미보_2016.xlsx
data/보_구미보_2017.xlsx
data/보_구미보_2018.xlsx
data/보_구미보_2019.xlsx


In [411]:
file_names = []
file_names = ['AWS_옥산_2016.xlsx', 'AWS_옥산_2017.xlsx', 'AWS_옥산_2018.xlsx', 'AWS_옥산_2019.xlsx']
folder = 'data'
aws_dfs = []
for file_name in file_names:
    path = os.path.join(folder, file_name)
    print(path)
    aws_dfs.append(pd.read_excel(path).iloc[:, -4:])
aws_df = pd.concat(aws_dfs, axis=0)

aws_mean = aws_df.mean()
aws_std = aws_df.std()
aws_df = (aws_df - aws_mean)/aws_std
aws_df.reset_index(drop=True, inplace=True)

data/AWS_옥산_2016.xlsx
data/AWS_옥산_2017.xlsx
data/AWS_옥산_2018.xlsx
data/AWS_옥산_2019.xlsx


In [29]:
folder = 'data'
path = os.path.join(folder, 'date-15-20.xlsx')
date_df = pd.read_excel(path, index_col='ymdh')
#date_df = pd.read_excel(path)

In [30]:
date_df

""
ymdh
2015-01-01 00:00
2015-01-01 01:00
2015-01-01 02:00
2015-01-01 03:00
2015-01-01 04:00
...
2020-11-15 19:00
2020-11-15 20:00
2020-11-15 21:00


In [197]:
file_names = [
              ['말지천_2015.xlsx',
               '말지천_2016.xlsx',
               '말지천_2017.xlsx',
               '말지천_2018.xlsx',
               '말지천_2019.xlsx',
               '말지천_2020.xlsx'],
              ['병성천-1_2015.xlsx',
               '병성천-1_2016.xlsx',
               '병성천-1_2017.xlsx',
               '병성천-1_2018.xlsx',
               '병성천-1_2019.xlsx',
               '병성천-1_2020.xlsx'],
              ['상주3_2015.xlsx',
               '상주3_2016.xlsx',
               '상주3_2017.xlsx',
               '상주3_2018.xlsx',
               '상주3_2019.xlsx',
               '상주3_2020.xlsx'],
              ['위천6_2015.xlsx',
               '위천6_2016.xlsx',
               '위천6_2017.xlsx',
               '위천6_2018.xlsx',
               '위천6_2019.xlsx',
               '위천6_2020.xlsx'],
             ]
folder = 'water_data/nak/총량'

# what's nh3n, no3n, ss, po4p, dtn, dtp?
#qm_cols = ['nh3n', 'no3n', 'ph', 'cod', 'ec', 'bod', 'ss', 'do', 'tn', 'tp', 'chlorophylla', 'toc']
tm_cols = ['iem_flux', 'iem_wtrtp',
       'iem_ph', 'iem_doc', 'iem_bod', 'iem_cod', 'iem_ss', 'iem_ec', 'iem_tn',
       'iem_tp', 
       'iem_chla', 'iem_toc']
#tm_cols = [ 'iem_ph',  'iem_chla']
#tm_cols = ['iem_chla']
#df = df.loc[:, qm_cols]

tm_dfs = []
tm_inter_dfs = []
for j in range(len(file_names)):
    dfs = []
    for i in range(len(file_names[j])):
        path = os.path.join(folder, file_names[j][i])
        dfs.append(pd.read_excel(path, index_col='dt'))
        dfs[i] = dfs[i].loc[:, tm_cols]
    tm_dfs.append(pd.concat(dfs))
    tm_dfs[j] = pd.concat([date_df, tm_dfs[j]], axis=1)
    tm_dfs[j].reset_index(drop=True, inplace=True)
    first_series = tm_dfs[j].apply(pd.Series.first_valid_index)
    last_series = tm_dfs[j].apply(pd.Series.first_valid_index)
    for col in first_series.index:
        tm_dfs[j].loc[0, col] = tm_dfs[j].loc[first_series[col], col]
        tm_dfs[j].loc[len(tm_dfs[j])-1, col] = tm_dfs[j].loc[last_series[col], col]
    
    tm_dfs[j] = tm_dfs[j].interpolate(method='pchip')

In [198]:
tm_mean = []
tm_std = []

for i in range(len(tm_dfs)):
    tm_mean.append(tm_dfs[i].mean())
    tm_std.append(tm_dfs[i].std())
    tm_dfs[i] = (tm_dfs[i]-tm_mean[i])/tm_std[i]
    tm_dfs[i].columns=tm_dfs[i].columns+'%d'%(i+1)

In [199]:
file_names = [
              ['병성천-1_2015.xlsx',
               '병성천-1_2016.xlsx',
               '병성천-1_2017.xlsx',
               '병성천-1_2018.xlsx',
               '병성천-1_2019.xlsx',
               '병성천-1_2020.xlsx'],
              ['상주2_2015.xlsx',
               '상주2_2016.xlsx',
               '상주2_2017.xlsx',
               '상주2_2018.xlsx',
               '상주2_2019.xlsx',
               '상주2_2020.xlsx'],
              ['상주3_2015.xlsx',
               '상주3_2016.xlsx',
               '상주3_2017.xlsx',
               '상주3_2018.xlsx',
               '상주3_2019.xlsx',
               '상주3_2020.xlsx'],
             ]
folder = 'water_data/nak/수질'

# what's nh3n, no3n, ss, po4p, dtn, dtp?
#qm_cols = ['nh3n', 'no3n', 'ph', 'cod', 'ec', 'bod', 'ss', 'do', 'tn', 'tp', 'chlorophylla', 'toc']
#qm_cols = ['ss', 'chlorophylla']
qm_cols = ['chlorophylla']
#df = df.loc[:, qm_cols]

qm_dfs = []
qm_inter_dfs = []
for j in range(len(file_names)):
    dfs = []
    for i in range(len(file_names[j])):
        path = os.path.join(folder, file_names[j][i])
        dfs.append(pd.read_excel(path, index_col='dt'))
        dfs[i] = dfs[i].loc[:, qm_cols]
    qm_dfs.append(pd.concat(dfs))
    qm_dfs[j] = pd.concat([date_df, qm_dfs[j]], axis=1)
    qm_dfs[j].reset_index(drop=True, inplace=True)
    first_series = qm_dfs[j].apply(pd.Series.first_valid_index)
    last_series = qm_dfs[j].apply(pd.Series.first_valid_index)
    for col in first_series.index:
        qm_dfs[j].loc[0, col] = qm_dfs[j].loc[first_series[col], col]
        qm_dfs[j].loc[len(qm_dfs[j])-1, col] = qm_dfs[j].loc[last_series[col], col]
    
    qm_dfs[j] = qm_dfs[j].interpolate(method='pchip')

In [200]:
qm_mean = []
qm_std = []

for i in range(len(qm_dfs)):
    qm_dfs[i].columns=qm_dfs[i].columns+'%d'%(i+1)
    qm_mean.append(qm_dfs[i].mean())
    qm_std.append(qm_dfs[i].std())
    qm_dfs[i] = (qm_dfs[i]-qm_mean[i])/qm_std[i]

In [109]:
mean = pd.concat([train_mean, bo_mean, aws_mean]+qm_mean)
std = pd.concat([train_std, bo_std, aws_std]+qm_std)
mean

NameError: name 'bo_mean' is not defined

In [280]:
gan_dfs[0][gan_dfs[1].columns]




KeyError: "None of [Index(['tmpr_value2', 'ph_value2', 'do_value2', 'ec_value2', 'toc_value2',\n       '총질소_값2', '총인_값2', '클로로필-a_값2'],\n      dtype='object')] are in the [columns]"

**Feature selection**

In [341]:
ori_df = pd.concat(ori_dfs, axis=1)
gan_df = pd.concat(gan_dfs, axis=1)
#ori_df = pd.concat(ori_dfs+[bo_df, aws_df]+qm_dfs, axis=1)
#gan_df = pd.concat(gan_dfs+[bo_df, aws_df]+qm_dfs, axis=1)
#ori_df = pd.concat(ori_dfs+qm_dfs+[tm_dfs[2]], axis=1)
#gan_df = pd.concat(gan_dfs+qm_dfs+[tm_dfs[2]], axis=1)
#ori_df = pd.concat([ori_dfs[0]]+qm_dfs+[tm_dfs[2]], axis=1)
#gan_df = pd.concat([gan_dfs[0]]+qm_dfs+[tm_dfs[2]], axis=1)
#ori_df = pd.concat([ori_dfs[0]], axis=1)
#gan_df = pd.concat([gan_dfs[0]], axis=1)
#ori_df = pd.concat(ori_dfs+[bo_df, aws_df], axis=1)
#gan_df = pd.concat(gan_dfs+[bo_df, aws_df], axis=1)

In [342]:
#ori_df = pd.concat(ori_dfs, axis=1)
#gan_df = pd.concat(gan_dfs, axis=1)

#ori_df = pd.concat(ori_dfs+[bo_df, aws_df]+qm_dfs, axis=1)
#gan_df = pd.concat(gan_dfs+[bo_df, aws_df]+qm_dfs, axis=1)

total_no = ori_df.shape[0]
train_no = int(total_no*0.7)
val_no = int(total_no*0.1)

train_slice = slice(0, train_no)
#train_slice = slice(val_no, val_no+train_no)
#val_slice = slice(train_no, train_no+val_no)
#val_slice = slice(train_no, None)
test_slice = slice(train_no, train_no+val_no)
#test_slice = slice(train_no+val_no, None)
#test_slice = slice(train_no+val_no, None)
#val_slice = slice(train_no+val_no, None)
#val_slice = slice(val_no+train_no, None)
val_slice = slice(train_no+val_no, None)

train_df = pd.DataFrame(gan_df[train_slice])
val_df = pd.DataFrame(gan_df[val_slice])
test_df = pd.DataFrame(gan_df[test_slice])

train_ori_df = pd.DataFrame(ori_df[train_slice])
val_ori_df = pd.DataFrame(ori_df[val_slice])
test_ori_df = pd.DataFrame(ori_df[test_slice])

num_features = train_df.shape[1]
num_features

20

In [70]:
len(train_df)

46216

In [175]:
gan_hr_df = gan_df.groupby(gan_df.index //24).mean()
total_no = gan_hr_df.shape[0]
train_no = int(total_no*0.7)

train_slice = slice(0, train_no)
val_slice = slice(train_no, None)
test_slice = slice(0, None)

train_day_df = gan_hr_df[train_slice]
val_day_df = gan_hr_df[val_slice]
test_day_df = gan_hr_df[test_slice]

In [290]:
train_df.columns

Index(['tmpr_value1', 'ph_value1', 'do_value1', 'ec_value1', 'toc_value1',
       '총질소_값1', '총인_값1', '클로로필-a_값1', 'Day sin1', 'Day cos1', 'Year sin1',
       'Year cos1'],
      dtype='object')

**OUT FEATURE**

In [343]:
#  2: DO 4: TOC, 5: TN, 6: TP, 7: Chl-a,
out_features = [7]
out_num_features=len(out_features)
input_days = 14 
out_feature_name = train_df.columns[out_features[0]]

**Data Generator**

In [159]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df=train_df, val_df=val_df, test_df=test_df,
            #train_df=None, val_df=None, test_df=None,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

In [160]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

In [161]:
def plot(self, model=None, plot_col='T (degC)', max_subplots=3):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]', fontproperties=fprop)
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
    else:
      label_col_index = plot_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                  marker='X', edgecolors='k', label='Predictions',
                  c='#ff7f0e', s=64)

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot = plot

In [162]:
@property
def train(self):
  return self.make_dataset(self.train_df, train=True)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example

In [163]:
class WaterDataGenerator(keras.utils.Sequence):
    'Generates data for water'
    def __init__(self,
                 imputed_data,
                 ori_data = None,
                 batch_size=32,
                 input_width=24*7,
                 label_width=24*3,
                 shift=24*3,
                 skip_time = None,
                 shuffle = True,
                 out_features = None,
                 out_num_features = None,
                ):
        'Initialization'
        self.window_size = input_width+shift
        self.total_no = imputed_data.shape[0]
        self.data = imputed_data
        self.input_width = input_width
        self.label_width = label_width
        self.batch_size = batch_size
        self.input_shape = (batch_size, input_width, self.data.shape[1])
        self.out_num_features = out_num_features
        if out_features:
            self.out_features = out_features
        else:
            self.out_features = [i for i in range(out_num_features)]
        self.label_shape = (batch_size, label_width, self.out_num_features)
        if (skip_time):
            # TO-DO
            self.no = self.total_no - self.window_size
            self.data_idx = np.arange(0, self.no)
        else:
            self.no = self.total_no - self.window_size
            self.data_idx = np.arange(0, self.no)
            
        if shuffle:
            self.batch_idx = np.random.permutation(self.no)
        else:
            self.batch_idx = np.arange(0, self.no)
        self.batch_id = 0
        #print('')
        #print('self.total_no =', self.total_no)
        #print('')
        
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        #return int(128/self.batch_size)
        #return 2
        return self.no//self.batch_size
        #return 1

    def __getitem__(self, index):
        'Generate one batch of data'
        #print('index =', index)
        #print('self.no =', self.no)
        #print('self.total_no =', self.total_no)
        #print('')
        #print('self.no =', self.no, 'self.batch_id =', self.batch_id)
        # Sample batch
        label_width = self.label_width
        batch_idx = self.batch_idx
        
        x = np.empty((0, self.input_width, self.data.shape[1]))
        y = np.empty((0, self.label_width, self.out_num_features))
        for cnt in range(0, self.batch_size):
            i = self.batch_id
            self.batch_id += 1
            idx1 = self.data_idx[batch_idx[i]]
            idx2 = idx1 + self.input_width
            
            X = self.data[idx1:idx2]
            
            idx1 = self.data_idx[batch_idx[i]] + self.window_size - label_width
            idx2 = idx1 + label_width
            
            #Y = self.data[idx1:idx2,:,:out_num_features]
            Y = self.data.iloc[idx1:idx2, self.out_features]
            #print('Y.shape = ', Y.shape)
            #Y = Y.iloc[:,:out_num_features]
            
            self.batch_id %= self.no
            
            x = np.append(x, [X], axis = 0)
            y = np.append(y, [Y], axis = 0)
            
        return x, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        return

In [164]:
def make_dataset_water(self, data, train=False):
  dg = WaterDataGenerator(
      data,
      batch_size = 128,
      #batch_size = 128 if train else data.shape[0]-(self.input_width+self.label_width),
      input_width = self.input_width,
      label_width = self.label_width,
      shift = self.shift,
      out_features = out_features,
      out_num_features = out_num_features,
  )
  #self.dg = dg
  ds = tf.data.Dataset.from_generator(
      lambda: dg,
      output_types=(tf.float32, tf.float32),
      output_shapes=(
        dg.input_shape,
        dg.label_shape
        #[batch_size, train_generator.dim],
        #[batch_size, train_generator.dim],
      )
  )
  #return ds.batch(batch_size=128)
  if train:
      #return ds.repeat(10).prefetch(3)
      return ds.repeat(-1).prefetch(5)
  else:
      return ds.prefetch(5)

WindowGenerator.make_dataset = make_dataset_water

In [165]:
def plot2(self, model=None, plot_col=0, max_subplots=3):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  #plot_out_col_index = self.column_indices[plot_out_col]
  out_features_np = np.array(out_features)
  plot_out_col_index = np.argwhere(out_features_np == plot_col_index)[0][0]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]', fontproperties=fprop)
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
      label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
    else:
      label_col_index = plot_col_index
      label_out_col_index = plot_out_col_index

    if label_col_index is None:
      continue

    plt.plot(self.label_indices, labels[n, :, label_out_col_index],
                label='Labels', c='#2ca02c')
    if model is not None:
      predictions = model(inputs)
      plt.plot(self.label_indices, predictions[n, :, label_out_col_index],
                  marker=None, label='Predictions',
                  c='#ff7f0e')

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot2 = plot2

In [166]:
train_df.columns

Index(['tmpr_value1', 'ph_value1', 'do_value1', 'ec_value1', 'toc_value1',
       '총질소_값1', '총인_값1', '클로로필-a_값1', 'Day sin1', 'Day cos1', 'Year sin1',
       'Year cos1', 'tmpr_value2', 'ph_value2', 'do_value2', 'ec_value2',
       'toc_value2', '총질소_값2', '총인_값2', '클로로필-a_값2', 'tmpr_value3',
       'ph_value3', 'do_value3', 'ec_value3', 'toc_value3', '총질소_값3', '총인_값3',
       '클로로필-a_값3'],
      dtype='object')

In [331]:
plt.figure()
plt.plot(gan_dfs[0].iloc[:, 6])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [332]:
plt.figure()
plt.plot(gan_dfs[0].iloc[:, 7])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [344]:
#OUT_STEPS = 24*5
OUT_STEPS = 24*14
SHIFT = 24*14
multi_window = WindowGenerator(input_width=24*input_days,
                               label_width=OUT_STEPS,
                               shift=SHIFT,
                               train_df=train_df,
                               val_df=val_df,
                               test_df=test_df,
                               )

#multi_window.plot2(plot_col=out_features[0])
#multi_window.plot2(plot_col='클로로필-a_값1', plot_out_col='클로로필-a_값1')
#multi_window.plot2(plot_col='클로로필-a_값1')
#multi_window.plot2(plot_col='총인_값1')
multi_window.plot2(plot_col=out_feature_name)
#multi_window.plot2(plot_col='toc_value1')
multi_window

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Total window size: 672
Input indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 

In [21]:
'''
OUT_STEPS = 5
multi_window = WindowGenerator(input_width=input_days,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS,
                               train_df=train_day_df,
                               val_df=val_day_df,
                               test_df=test_day_df,
                               )

#multi_window.plot2(plot_col=out_features[0])
#multi_window.plot2(plot_col='클로로필-a_값1', plot_out_col='클로로필-a_값1')
multi_window.plot2(plot_col='클로로필-a_값1')
multi_window
'''

"\nOUT_STEPS = 5\nmulti_window = WindowGenerator(input_width=input_days,\n                               label_width=OUT_STEPS,\n                               shift=OUT_STEPS,\n                               train_df=train_day_df,\n                               val_df=val_day_df,\n                               test_df=test_day_df,\n                               )\n\n#multi_window.plot2(plot_col=out_features[0])\n#multi_window.plot2(plot_col='클로로필-a_값1', plot_out_col='클로로필-a_값1')\nmulti_window.plot2(plot_col='클로로필-a_값1')\nmulti_window\n"

In [170]:
multi_val_performance = {}
multi_performance = {}

In [56]:
class MultiStepLastBaseline(tf.keras.Model):
  def call(self, inputs):
    #print(inputs[:, -1:, 0:1])
    #return tf.tile(inputs[:, -1:, :out_num_features], [1, OUT_STEPS, 1])
    return tf.tile(inputs[:, -1:, (out_features[0]):(out_features[0]+1)], [1, OUT_STEPS, 1])
    #return tf.tile(inputs[:, -1:, out_features[0]:(out_features[1]+1)], [1, OUT_STEPS, 1])

last_baseline = MultiStepLastBaseline()
last_baseline.compile(loss=tf.losses.MeanSquaredError(),
                      metrics=[tf.metrics.MeanAbsoluteError()])

#multi_val_performance = {}
#multi_performance = {}

multi_val_performance['Last'] = last_baseline.evaluate(multi_window.val)
multi_performance['Last'] = last_baseline.evaluate(multi_window.test)
print('val performance =', multi_val_performance['Last'])
print('test performance = ', multi_performance['Last'])
multi_window.plot2(last_baseline, plot_col=train_df.columns[out_features[0]])

37/37 [==============================] - 4s 119ms/step - loss: 0.4109 - mean_absolute_error: 0.4645
val performance = [0.2490123212337494, 0.34261825680732727]
test performance =  [0.4108981490135193, 0.4644666612148285]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [233]:
#MAX_EPOCHS = 400
MAX_EPOCHS = 20

def nse(y_true, y_pred):
    mean = tf.reduce_mean(y_true)
    return 1. - tf.reduce_sum(tf.square(y_true-y_pred))/tf.reduce_sum(tf.square(y_true-mean))

def compile_and_fit(model, window, patience=1000):
  checkpoint = keras.callbacks.ModelCheckpoint('save'+'/best_model.h5', monitor='val_loss', verbose=1,
    save_best_only=True, mode='auto', period=1)
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError(), nse])
  #model.compile(loss=GAIN.RMSE_loss)

  history = model.fit(window.train, epochs=MAX_EPOCHS, steps_per_epoch=10,
                      validation_data=window.val,
                      callbacks=[early_stopping, checkpoint])
  return history

In [24]:
def plot_history(history):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(history.history['loss'], label='loss')
    ax.plot(history.history['mean_absolute_error'], label='mae')
    ax.plot(history.history['val_loss'], label='val_loss')
    ax.plot(history.history['val_mean_absolute_error'], label='val_mae')
    #plt.legend(history.history.keys(), loc='upper right')
    #ax.legend(loc='upper center')
    ax.legend()
    ax.set_xlabel("epochs")
    ax.set_ylabel("loss")
    plt.show()

In [ ]:
import IPython
multi_linear_model = tf.keras.Sequential([
    # Take the last time-step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.Lambda(lambda x: x[:, -1:, :]),
    # Shape => [batch, 1, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

history = compile_and_fit(multi_linear_model, multi_window)

#IPython.display.clear_output()
#multi_val_performance['Linear'] = multi_linear_model.evaluate(multi_window.val.repeat(-1), steps=100)
#multi_performance['Linear'] = multi_linear_model.evaluate(multi_window.test.repeat(-1), verbose=1, steps=100)
multi_val_performance['Linear'] = multi_linear_model.evaluate(multi_window.val)
multi_performance['Linear'] = multi_linear_model.evaluate(multi_window.test, verbose=1)
#multi_window.plot(multi_linear_model, plot_col=0)
print('val performance =', multi_val_performance['Linear'])
print('test performance = ', multi_performance['Linear'])

In [ ]:
len(train_df)//128

In [ ]:
plot_history(history)

In [ ]:
multi_window.plot2(multi_linear_model, plot_col=out_feature_name)

In [ ]:
58*24*10

In [ ]:
10240/2

In [ ]:
MAX_EPOCHS = 40 
multi_dense_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.Flatten(),
    # Shape => [batch, 1, dense_units]
    #tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(8192, activation='relu'),
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dense(2048, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    # Shape => [batch, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

history = compile_and_fit(multi_dense_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['Dense'] = multi_dense_model.evaluate(multi_window.val)
multi_performance['Dense'] = multi_dense_model.evaluate(multi_window.test, verbose=1)
print('val performance =', multi_val_performance['Dense'])
print('test performance = ', multi_performance['Dense'])

In [ ]:
plot_history(history)

In [ ]:
multi_window.plot2(multi_dense_model, plot_col=out_feature_name)

In [ ]:
MAX_EPOCHS = 5 
#CONV_LAYER_NO = 3
CONV_WIDTH = 3
multi_conv_model = tf.keras.Sequential([
    keras.layers.Conv1D(256, activation='linear', kernel_size=(CONV_WIDTH)),
    #keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Conv1D(256, activation='linear', kernel_size=(CONV_WIDTH)),
    #keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.MaxPooling1D(padding='same'),
    keras.layers.Conv1D(512, activation='linear', kernel_size=(CONV_WIDTH)),
    #keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Conv1D(512, activation='linear', kernel_size=(CONV_WIDTH)),
    #keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Conv1D(512, activation='linear', kernel_size=(CONV_WIDTH)),
    #keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.MaxPooling1D(padding='same'),
    keras.layers.Conv1D(1024, activation='linear', kernel_size=(CONV_WIDTH)),
    #keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Conv1D(1024, activation='linear', kernel_size=(CONV_WIDTH)),
    #keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Conv1D(1024, activation='linear', kernel_size=(CONV_WIDTH)),
    #keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.MaxPooling1D(padding='same'),
    keras.layers.Conv1D(2048, activation='linear', kernel_size=(CONV_WIDTH)),
    #keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Conv1D(2048, activation='linear', kernel_size=(CONV_WIDTH)),
    #keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Conv1D(2048, activation='linear', kernel_size=(CONV_WIDTH)),
    #keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    #keras.layers.MaxPooling1D(padding='same'),
    
    #keras.layers.Conv1D(4096, activation='linear', kernel_size=(CONV_WIDTH)),
    #keras.layers.BatchNormalization(),
    #keras.layers.Activation('relu'),
    #keras.layers.Conv1D(4096, activation='linear', kernel_size=(CONV_WIDTH)),
    #keras.layers.BatchNormalization(),
    #keras.layers.Activation('relu'),
    #keras.layers.MaxPooling1D(padding='same'),
    tf.keras.layers.Flatten(),
    #keras.layers.MaxPooling1D(pool_size=2),
    #keras.layers.Conv1D(1024, activation='relu', kernel_size=(CONV_WIDTH)),
    #keras.layers.MaxPooling1D(pool_size=2),
    #keras.layers.Conv1D(1024, activation='relu', kernel_size=(CONV_WIDTH)),
    #keras.layers.MaxPooling1D(pool_size=2),
    #keras.layers.Conv1D(1024, activation='relu', kernel_size=(CONV_WIDTH)),
    keras.layers.Dropout(0.5),
    
    # Shape => [batch, 1,  out_steps*features]
    #tf.keras.layers.Dense(OUT_STEPS*num_features,
                          #kernel_initializer=tf.initializers.zeros),
    tf.keras.layers.Dense(OUT_STEPS*num_features),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, num_features])
])



In [ ]:
history = compile_and_fit(multi_conv_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['Conv'] = multi_conv_model.evaluate(multi_window.val)
multi_performance['Conv'] = multi_conv_model.evaluate(multi_window.test, verbose=1)
print('val performance =', multi_val_performance['Conv'])
print('test performance = ', multi_performance['Conv'])

In [ ]:
plot_history(history)

**GRU model**  
best: all feature, 10 days, 256,256
```
val performance = [0.46297532320022583, 0.4968714416027069]
test performance =  [0.5573816895484924, 0.5664846301078796]
```

best: aws,bo feature, 5 days, 256,256 (dropout 0.1, 0.5) dense dropout 0.1
10%, 80%, 10%(val)
```
val performance = [0.5091502666473389, 0.5460899472236633, 0.5668533444404602]
test performance =  [0.27155840396881104, 0.3736426532268524, 0.59783536195755]
```

In [ ]:
#MAX_EPOCHS = 150
MAX_EPOCHS = 20 
multi_gru_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units]
    # Adding more `lstm_units` just overfits more quickly.
    #tf.keras.layers.LSTM(32, return_sequences=False),
    #tf.keras.layers.GRU(256, return_sequences=True, dropout=0.1, recurrent_dropout=0.5),
    #tf.keras.layers.GRU(256, return_sequences=False, dropout=0.1, recurrent_dropout=0.5),
    tf.keras.layers.GRU(256, return_sequences=True, dropout=0.1, recurrent_dropout=0.5),
    tf.keras.layers.GRU(256, return_sequences=False, dropout=0.1, recurrent_dropout=0.5),
    # Shape => [batch, out_steps*features]
    tf.keras.layers.Dropout(0.5),
    #tf.keras.layers.Dense(OUT_STEPS*out_num_features*2, activation='relu'),
    #tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(OUT_STEPS*out_num_features),
    #tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          #kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

history = compile_and_fit(multi_gru_model, multi_window)
#multi_gru_model.load_weights('nagdong_cl.h5')

#IPython.display.clear_output()

multi_val_performance['GRU'] = multi_gru_model.evaluate(multi_window.val)
multi_performance['GRU'] = multi_gru_model.evaluate(multi_window.test, verbose=1)
print('val performance =', multi_val_performance['GRU'])
print('test performance = ', multi_performance['GRU'])

Epoch 1/20
10/10 [==============================] - 16s 1s/step - loss: 0.9166 - mean_absolute_error: 0.7243 - nse: -0.0221 - val_loss: 0.9048 - val_mean_absolute_error: 0.7026 - val_nse: 0.2076

Epoch 00001: val_loss improved from inf to 0.90484, saving model to save/best_model.h5
Epoch 2/20
10/10 [==============================] - 14s 1s/step - loss: 0.7604 - mean_absolute_error: 0.6207 - nse: 0.2252 - val_loss: 0.8031 - val_mean_absolute_error: 0.6556 - val_nse: 0.2967

Epoch 00002: val_loss improved from 0.90484 to 0.80310, saving model to save/best_model.h5
Epoch 3/20
10/10 [==============================] - 14s 1s/step - loss: 0.7476 - mean_absolute_error: 0.5932 - nse: 0.3145 - val_loss: 0.8101 - val_mean_absolute_error: 0.6561 - val_nse: 0.2894

Epoch 00003: val_loss did not improve from 0.80310
Epoch 4/20
10/10 [==============================] - 14s 1s/step - loss: 0.6512 - mean_absolute_error: 0.5611 - nse: 0.3012 - val_loss: 0.7608 - val_mean_absolute_error: 0.6371 - val_nse

In [322]:
multi_gru_model.load_weights('save/best_model.h5')
multi_val_performance['GRU'] = multi_gru_model.evaluate(multi_window.val)
multi_performance['GRU'] = multi_gru_model.evaluate(multi_window.test, verbose=1)
print('val performance =', multi_val_performance['GRU'])
print('test performance = ', multi_performance['GRU'])

48/48 [==============================] - 6s 127ms/step - loss: 0.3964 - mean_absolute_error: 0.4475 - nse: 0.4719
val performance = [0.3772953152656555, 0.43701237440109253, 0.3745199739933014]
test performance =  [0.39642369747161865, 0.44746726751327515, 0.4718872606754303]


In [294]:
plot_history(history)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [295]:
multi_window.plot2(multi_gru_model, plot_col=out_feature_name)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [76]:
multi_gru_model.save_weights('nagdong_cl.h5')

In [238]:
#MAX_EPOCHS = 150
MAX_EPOCHS = 15
multi_lstm_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units]
    # Adding more `lstm_units` just overfits more quickly.
    #tf.keras.layers.LSTM(32, return_sequences=False),
    #tf.keras.layers.LSTM(256, return_sequences=True, dropout=0.1, recurrent_dropout=0.5),
    #tf.keras.layers.LSTM(256, return_sequences=True, dropout=0.1, recurrent_dropout=0.5),
    #tf.keras.layers.LSTM(128, return_sequences=True, dropout=0.1, recurrent_dropout=0.5),
    tf.keras.layers.GRU(256, return_sequences=True, dropout=0.1, recurrent_dropout=0.5),
    tf.keras.layers.GRU(128, return_sequences=True, dropout=0.1, recurrent_dropout=0.5),
    tf.keras.layers.GRU(64, return_sequences=False, dropout=0.1, recurrent_dropout=0.5),
    # Shape => [batch, out_steps*features]
    tf.keras.layers.Dropout(0.5),
    #tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          #kernel_initializer=tf.initializers.zeros),
    tf.keras.layers.Dense(OUT_STEPS*out_num_features),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

history = compile_and_fit(multi_lstm_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.val)
multi_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.test)
print('val performance =', multi_val_performance['LSTM'])
print('test performance = ', multi_performance['LSTM'])

Epoch 1/15
10/10 [==============================] - 20s 2s/step - loss: 0.4384 - mean_absolute_error: 0.5326 - nse: -0.2803 - val_loss: 0.1698 - val_mean_absolute_error: 0.3251 - val_nse: -0.2159

Epoch 00001: val_loss improved from inf to 0.16979, saving model to save/best_model.h5
Epoch 2/15
10/10 [==============================] - 16s 2s/step - loss: 0.3043 - mean_absolute_error: 0.4247 - nse: 0.1351 - val_loss: 0.1125 - val_mean_absolute_error: 0.2611 - val_nse: 0.1954

Epoch 00002: val_loss improved from 0.16979 to 0.11251, saving model to save/best_model.h5
Epoch 3/15
10/10 [==============================] - 16s 2s/step - loss: 0.2198 - mean_absolute_error: 0.3560 - nse: 0.3406 - val_loss: 0.0828 - val_mean_absolute_error: 0.2206 - val_nse: 0.4075

Epoch 00003: val_loss improved from 0.11251 to 0.08282, saving model to save/best_model.h5
Epoch 4/15
10/10 [==============================] - 16s 2s/step - loss: 0.1756 - mean_absolute_error: 0.3163 - nse: 0.4706 - val_loss: 0.0984 - 

In [239]:
plot_history(history)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [240]:
multi_window.plot2(multi_lstm_model, plot_col=out_feature_name)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
x = np.arange(len(multi_performance))
width = 0.3


metric_name = 'mean_absolute_error'
metric_index = multi_gru_model.metrics_names.index('mean_absolute_error')
val_mae = [v[metric_index] for v in multi_val_performance.values()]
test_mae = [v[metric_index] for v in multi_performance.values()]

plt.figure()
plt.bar(x - 0.17, val_mae, width, label='Validation')
plt.bar(x + 0.17, test_mae, width, label='Test')
plt.xticks(ticks=x, labels=multi_performance.keys(),
           rotation=45)
plt.ylabel(f'MAE (average over all times and outputs)')
_ = plt.legend()
plt.show()

**Daily Prediction**

In [323]:
from tqdm import tqdm

class DaysPrediction(object):
    def __init__(self,
                 df=test_df,
                 in_steps=24*input_days,
                 out_steps=OUT_STEPS,
                 shift=SHIFT,
                ):
        self.df = df
        self.in_steps = in_steps
        self.out_steps = out_steps
        # fix해야함
        #len_tt = df.shape[0]//in_steps*in_steps
        #self.n_days = (len_tt - in_steps)//24
        self.n_days = df.shape[0]//24
        self.pred_days = out_steps//24
        self.start_day = (SHIFT-OUT_STEPS)//24
        self.in_days = in_steps//24
        self.gt = np.full((self.n_days), np.nan)
        for i in range(self.n_days):
            x = self.df[i*24:(i*24+24)].to_numpy()
            #x = val_df[i*24:(i*24+24)].to_numpy()
            self.gt[i] = np.average(x[0:24, out_features[0]])
        
    def predict(self, model):
        preds = np.full((self.start_day+self.pred_days, self.n_days), np.nan)
        x_batch = np.empty(((self.n_days-self.in_days)*24, self.in_steps, self.df.shape[1]))
        for i in range(self.n_days-self.in_days):
            x = self.df[i*24:(i*24+self.in_steps)].to_numpy()
            #x = val_df[i*24:(i*24+self.in_steps)].to_numpy()
            x = x.reshape((-1, self.in_steps, self.df.shape[1]))
            x_batch[i*24:(i+1)*24] = x
            
        #print('x_batch.shape =', x_batch.shape)
        ds = tf.data.Dataset.from_tensor_slices(x_batch).batch(256)
        y_batch = model.predict(ds, verbose=1)
        #print('y_batch.shape =', y_batch.shape)
        for i in range(self.n_days-self.in_days):
            y = y_batch[i*24:(i+1)*24]
            for j in range(self.pred_days):
                if i+self.in_days+j+self.start_day >= self.n_days:
                    break
                preds[j+self.start_day][i+self.in_days+j+self.start_day] = np.average(y[:, j*24:j*24+24, :])
        return preds

In [345]:
SHIFT, OUT_STEPS


(144, 144)

In [471]:
class Prediction(object):
    def __init__(self,
                 df=test_df,
                 in_steps=input_days,
                 out_steps=OUT_STEPS,
                ):
        self.df = df
        self.in_steps = in_steps
        self.out_steps = out_steps
        len_tt = df.shape[0]
        #self.n_days = len_tt - in_steps
        self.n_days = len_tt
        self.pred_days = out_steps
        self.in_days = in_steps
        self.gt = self.df.iloc[:, out_features[0]].to_numpy()
        
    def predict(self, model):
        preds = np.full((self.pred_days, self.n_days), np.nan)
        for i in tqdm(range(self.n_days-self.in_days)):
        #for i in tqdm(range(1)):
            x = self.df[i:(i+self.in_steps)].to_numpy()
            #x = val_df[i*24:(i*24+self.in_steps)].to_numpy()
            x = x.reshape((-1, self.in_steps, self.df.shape[1]))
            y = model.predict(x)
            #print(y.shape)
            for j in range(self.pred_days):
                if i+self.in_days+j >= self.n_days:
                    break
                preds[j][i+self.in_days+j] = y[0, j, 0]
        return preds

In [543]:
class HoursPrediction(object):
    def __init__(self,
                 df=test_df,
                 in_steps=24*input_days,
                 out_steps=OUT_STEPS,
                ):
        self.df = df
        self.in_steps = in_steps
        self.out_steps = out_steps
        # fix해야함
        #len_tt = df.shape[0]//in_steps*in_steps
        #self.n_days = (len_tt - in_steps)//24
        self.n_days = df.shape[0]//24
        self.pred_days = out_steps//24
        self.in_days = in_steps//24
        self.gt = self.df.iloc[:self.n_days*24, out_features[0]].to_numpy()
        #print('gt.shape =', self.gt.shape)
        #print('n_days =', self.n_days)
        #print('n_days*24 =', self.n_days*24)
        
    def predict(self, model):
        preds = np.full((self.pred_days, self.n_days*24), np.nan)
        #x_batch = np.empty((0, self.in_steps, self.df.shape[1]))
        for i in tqdm(range(self.n_days-self.in_days)):
            x = self.df[i*24:(i*24+self.in_steps)].to_numpy()
            #x = val_df[i*24:(i*24+self.in_steps)].to_numpy()
            x = x.reshape((-1, self.in_steps, self.df.shape[1]))
            y = model.predict(x)
            for j in range(self.pred_days):
                if i+self.in_days+j >= self.n_days:
                    break
                preds[j][(i+self.in_days+j)*24:(i+self.in_days+j+1)*24] = y[0, j*24:j*24+24, 0]

        return preds

In [29]:
def denormalize(data):
    return data*train_std[out_features[0]]+train_mean[out_features[0]]

In [30]:
preds_train = {}
preds_val = {}
preds_test = {}

**Daily avaraged prediction**

In [324]:
days_prediction_train = DaysPrediction(df=train_df)
days_prediction_val = DaysPrediction(df=val_df)
days_prediction_test = DaysPrediction(df=test_df)
daily = True

gt_train = denormalize(days_prediction_train.gt)
gt_val = denormalize(days_prediction_val.gt)
gt_test = denormalize(days_prediction_test.gt)

**Hourly prediction**

In [435]:
days_prediction_train = HoursPrediction(df=train_df)
days_prediction_val = HoursPrediction(df=val_df)
days_prediction_test = HoursPrediction(df=test_df)
daily = False

gt_train = denormalize(days_prediction_train.gt)
gt_val = denormalize(days_prediction_val.gt)
gt_test = denormalize(days_prediction_test.gt)

**Daily avaraged Model(warning)**

In [ ]:
days_prediction_train = Prediction(df=train_day_df)
days_prediction_val = Prediction(df=val_day_df)
days_prediction_test = Prediction(df=test_day_df)
daily = True

gt_train = denormalize(days_prediction_train.gt)
gt_val = denormalize(days_prediction_val.gt)
gt_test = denormalize(days_prediction_test.gt)

**Predict**

In [325]:
#preds_dict = {}
#days_prediction = DaysPrediction()
preds_train['GRU'] = denormalize(days_prediction_train.predict(multi_gru_model))
preds_val['GRU'] = denormalize(days_prediction_val.predict(multi_gru_model))
preds_test['GRU'] = denormalize(days_prediction_test.predict(multi_gru_model))

25/25 [==============================] - 2s 92ms/step


In [493]:
preds_val['GRU'][4][0:30]

array([       nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan, 8.64702736,
       9.10766748, 9.31373713, 9.51382185, 9.65105033, 8.93027386,
       8.78962696, 8.73516727, 8.87070062, 9.16555979, 8.54188793,
       8.46553372, 8.69801248, 8.49381289, 8.45470907, 8.47526382])

In [243]:
preds_train['LSTM'] = denormalize(days_prediction_train.predict(multi_lstm_model))
preds_val['LSTM'] = denormalize(days_prediction_val.predict(multi_lstm_model))
preds_test['LSTM'] = denormalize(days_prediction_test.predict(multi_lstm_model))

24/24 [==============================] - 3s 128ms/step


In [ ]:
preds_train['Dense'] = denormalize(days_prediction_train.predict(multi_dense_model))
preds_val['Dense'] = denormalize(days_prediction_val.predict(multi_dense_model))
preds_test['Dense'] = denormalize(days_prediction_test.predict(multi_dense_model))

In [326]:
pred_day=4

In [327]:
plt.figure(figsize=(8, 3))
plt.plot(gt_train)
plt.plot(preds_train['GRU'][pred_day])
#plt.plot(preds_train['LSTM'][pred_day])
#plt.plot(preds_train['Dense'][pred_day])
plt.show()
plt.figure(figsize=(8, 3))
plt.plot(gt_val)
plt.plot(preds_val['GRU'][pred_day])
#plt.plot(preds_val['LSTM'][pred_day])
#plt.plot(preds_val['Dense'][pred_day])
plt.show()
plt.figure(figsize=(8, 3))
plt.plot(gt_test)
plt.plot(preds_test['GRU'][pred_day])
#plt.plot(preds_test['LSTM'][pred_day])
#plt.plot(preds_test['Dense'][pred_day])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [301]:
def daily_performance_metric(gt, pred):
    mae = np.mean(np.absolute(gt - pred))
    mape = np.mean(np.absolute(gt - pred)/gt)
    mean = np.mean(gt)
    nse = 1.-np.sum((gt - pred)**2) /  np.sum((gt - mean)**2)
    return { 'mae': mae, 'nse': nse}

In [302]:
daily_train_performance = {}
daily_val_performance = {}
daily_test_performance = {}

In [328]:
#model_names = ['GRU', 'LSTM', 'Dense']
model_names = ['GRU']
#model_names = ['GRU', 'LSTM']
prefix = 'awsbo_'

if daily == True:
    pred_slice = slice(days_prediction_val.in_days+pred_day, None)
else:
    pred_slice = slice((days_prediction_val.in_days+pred_day)*24, None)

for model_name in model_names:
    daily_train_performance[prefix+model_name] = daily_performance_metric(gt_train[pred_slice], preds_train[model_name][pred_day][pred_slice])
    daily_val_performance[prefix+model_name] = daily_performance_metric(gt_val[pred_slice], preds_val[model_name][pred_day][pred_slice])
    daily_test_performance[prefix+model_name] = daily_performance_metric(gt_test[pred_slice], preds_test[model_name][pred_day][pred_slice])
print(daily_train_performance[prefix+'GRU']['nse'])
print(daily_test_performance[prefix+'GRU']['nse'])
print(daily_val_performance[prefix+'GRU']['nse'])
#print(daily_train_performance[prefix+'LSTM']['nse'])
#print(daily_test_performance[prefix+'LSTM']['nse'])
#print(daily_val_performance[prefix+'LSTM']['nse'])

0.37564839568161756
0.30499512990033206
0.16201144808644163


In [329]:
daily_test_performance[prefix+'GRU']['nse']

0.30499512990033206

In [330]:
daily_val_performance[prefix+'GRU']['nse']

0.16201144808644163

In [528]:
x = np.arange(len(daily_test_performance))
width = 0.3


#metric_name = 'mean_absolute_error'
#metric_index = multi_linear_model.metrics_names.index('mean_absolute_error')
metric_index = 'mae'
#metric_index = 'mae'
train_mae = [v[metric_index] for v in daily_train_performance.values()]
val_mae = [v[metric_index] for v in daily_val_performance.values()]
test_mae = [v[metric_index] for v in daily_test_performance.values()]
#test_mae = [v[metric_index] for v in multi_performance.values()]

plt.figure()
plt.bar(x - 0.32, train_mae, width, label='Train')
plt.bar(x, val_mae, width, label='Validation')
plt.bar(x + 0.32, test_mae, width, label='Test')
plt.xticks(ticks=x, labels=daily_test_performance.keys(),
           rotation=45)
plt.ylabel(f'MAE (average over all times and outputs)')
_ = plt.legend()
plt.show()


#metric_name = 'mean_absolute_error'
#metric_index = multi_linear_model.metrics_names.index('mean_absolute_error')
metric_index = 'nse'
#metric_index = 'mae'
train_nse = [v[metric_index] for v in daily_train_performance.values()]
val_nse = [v[metric_index] for v in daily_val_performance.values()]
test_nse = [v[metric_index] for v in daily_test_performance.values()]
#test_mae = [v[metric_index] for v in multi_performance.values()]

plt.figure()
plt.bar(x - 0.32, train_nse, width, label='Train')
plt.bar(x, val_nse, width, label='Validation')
plt.bar(x + 0.32, test_nse, width, label='Test')
plt.xticks(ticks=x, labels=daily_test_performance.keys(),
           rotation=45)
plt.ylabel(f'NSE (average over all times and outputs)')
_ = plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
pred_day = 4
pred_slice = slice(days_prediction.in_days+pred_day, None)

In [ ]:
performance = {}

In [ ]:
pred_day = 4
pred_slice = slice(days_prediction.in_days+pred_day, None)

mae, mape, nse = daily_performance_metric(gt[pred_slice], denorm_preds['GRU'][pred_day][pred_slice])
print('mae =', mae, 'mape =', mape, 'nse =', nse)
mae, mape, nse = daily_performance_metric(gt[pred_slice], denorm_preds['LSTM'][pred_day][pred_slice])
print('mae =', mae, 'mape =', mape, 'nse =', nse)

In [ ]:
pred_day = 0
pred_slice = slice(days_prediction.in_days+pred_day, None)

mae, mape, nse = daily_performance_metric(gt[pred_slice], denorm_preds['GRU'][pred_day][pred_slice])
print('mae =', mae, 'mape =', mape, 'nse =', nse)
mae, mape, nse = daily_performance_metric(gt[pred_slice], denorm_preds['LSTM'][pred_day][pred_slice])
print('mae =', mae, 'mape =', mape, 'nse =', nse)

In [ ]:
pred_day = 0
pred_slice = slice(days_prediction.in_days+pred_day, None)

mae, mape, nse = daily_performance_metric(gt[pred_slice], denorm_preds['GRU'][pred_day][pred_slice])
print('mae =', mae, 'mape =', mape, 'nse =', nse)
mae, mape, nse = daily_performance_metric(gt[pred_slice], denorm_preds['LSTM'][pred_day][pred_slice])
print('mae =', mae, 'mape =', mape, 'nse =', nse)

In [ ]:
pred_day = 0
pred_slice = slice(days_prediction.in_days+pred_day, None)

mae = np.mean(np.absolute(gt[pred_slice] - (denorm_preds['GRU'][pred_day])[pred_slice]))
print(mae)
mape = np.mean(np.absolute((gt[pred_slice] - denorm_preds['GRU'][pred_day][pred_slice])/gt[pred_slice]))
print(mape)
mean = np.mean(gt[pred_slice])
nse = 1.-np.sum((gt[pred_slice] - denorm_preds['GRU'][pred_day][pred_slice])**2) /  np.sum((gt[pred_slice] - mean)**2)
print('nse =', nse)

In [ ]:
pred_day = 0
pred_slice = slice(days_prediction.in_days+pred_day, None)

mae = np.mean(np.absolute(gt[pred_slice] - (denorm_preds['LSTM'][pred_day])[pred_slice]))
print(mae)
mape = np.mean(np.absolute((gt[pred_slice] - denorm_preds['LSTM'][pred_day][pred_slice])/gt[pred_slice]))
print(mape)
mean = np.mean(gt[pred_slice])
nse = 1.-np.sum((gt[pred_slice] - denorm_preds['LSTM'][pred_day][pred_slice])**2) /  np.sum((gt[pred_slice] - mean)**2)
print('nse =', nse)

In [ ]:
model = tf.keras.models.Sequential()

In [ ]:
width=24*10
#width=3
channels=8
kernel_size=3
in_data = keras.layers.Input(shape=(width, channels))
x = in_data
x = keras.layers.Conv1D(32, kernel_size=(kernel_size,), strides=1)(x)
x = keras.layers.Conv1D(32, kernel_size=(kernel_size,), strides=1)(x)
x = keras.layers.MaxPooling1D(padding='same')(x)
x = keras.layers.Conv1D(32, kernel_size=(kernel_size,), strides=1)(x)
x = keras.layers.Conv1D(32, kernel_size=(kernel_size,), strides=1)(x)
x = keras.layers.MaxPooling1D(padding='same')(x)
x = keras.layers.Conv1D(32, kernel_size=(kernel_size,), strides=1)(x)
x = keras.layers.Conv1D(32, kernel_size=(kernel_size,), strides=1)(x)
x = keras.layers.MaxPooling1D(padding='same')(x)
x = keras.layers.Conv1D(32, kernel_size=(kernel_size,), strides=1)(x)
x = keras.layers.Conv1D(32, kernel_size=(kernel_size,), strides=1)(x)
x = keras.layers.MaxPooling1D(padding='same')(x)
x = keras.layers.Conv1D(32, kernel_size=(kernel_size,), strides=1)(x)
x = keras.layers.Conv1D(32, kernel_size=(kernel_size,), strides=1)(x)
x = keras.layers.MaxPooling1D(padding='same')(x)
#out_data = keras.layers.Conv1D(32, kernel_size=(kernel_size,), strides=1)(x)
out_data = x

model = keras.models.Model(inputs=in_data, outputs=out_data)
model.summary()

In [ ]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [ ]:
8-7+1

In [ ]:
128*150

In [ ]:
train_df.shape

# 장기예측 (14일)